In [4]:
from collections import defaultdict
import heapq
import numpy as np

dx = [0,1,0,-1]
dy = [1,0,-1,0]

def calc_norm(a,b):
    return np.linalg.norm(np.array(a)-np.array(b))

def find_next(pos, board, visit_map, cost_map):
    # print(cost_map)
    shape = board.shape
    next_pos = []
    for i in range(4):
        x = pos[0]+dx[i]
        y = pos[1]+dy[i]
        if x < 0 or x>= shape[0] or y < 0 or y >= shape[1] or visit_map[x,y] == True:
            continue
        # print(cost_map[x,y], cost_map[pos])
        if cost_map[x,y] > (cost_map[pos]+1):
            cost_map[x,y] = cost_map[pos]+1
            next_pos.append((cost_map[x,y], x,y))
    return next_pos

def dijkstra(start, end, board):
    max_dir = board.shape[0] * board.shape[1]
    cost_map = np.array([[max_dir for j in range(board.shape[1])] for i in range(board.shape[0])])
    visit_map = np.array([[False for j in range(board.shape[1])] for i in range(board.shape[0])])
    visit_map[start] = True
    cost_map[start] = 0

    hq = []
    for nm in find_next(start, board, visit_map, cost_map):
        heapq.heappush(hq, nm)
    heapq.heapify(hq)
    while hq:
        cur_cost, curx, cury = heapq.heappop(hq)
        # print(cur_cost, curx, cury)
        visit_map[curx, cury] = True
        if visit_map[end]:
            break
        for nd in find_next((curx,cury), board, visit_map, cost_map):
            heapq.heappush(hq, nd)
        yield visit_map
    yield visit_map


def astar(start, end, board):
    max_dir = board.shape[0] * board.shape[1]
    cost_map = np.array([[max_dir for j in range(board.shape[1])] for i in range(board.shape[0])])
    # visit_map = np.array([[False for j in range(board.shape[1])] for i in range(board.shape[0])])
    visit_map = np.copy(board)
    visit_map[start] = True
    cost_map[start] = 0
    parent_map = {}

    hq = []
    for nm in find_next(start, board, visit_map, cost_map):
        heapq.heappush(hq, nm)
    heapq.heapify(hq)
    while hq:
        cur_cost, curx, cury = heapq.heappop(hq)
        # print(cur_cost, curx, cury)
        visit_map[curx, cury] = True
        if visit_map[end]:
            break
        for nc, nx, ny in find_next((curx,cury), board, visit_map, cost_map):
            if nc + calc_norm(end,(nx,ny)) > calc_norm(end, (curx,cury)) + cost_map[curx,cury] + 1:
                continue
            heapq.heappush(hq, (nc, nx, ny))
            parent_map[(nx, ny)] = (curx, cury)
        # print(hq)
        yield visit_map
    yield visit_map


In [7]:
import tkinter as tk
import threading
import random

root = tk.Tk()
root.title('test')

# Initialize game parameters
rows, cols, num_mines = 20, 20, 20
cell_size = 20
canvas_width, canvas_height = cols * cell_size, rows * cell_size

end_pos = (13,7)

# Create the canvas for the game grid

def make_board():
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    my_board = np.array([ [False for j in range(cols)] for i in range(rows)])
    cells = np.zeros((rows, cols), dtype="int")
    for row in range(rows):
        for col in range(cols):
            x0, y0 = col * cell_size, row * cell_size
            x1, y1 = x0 + cell_size, y0 + cell_size
            cell = canvas.create_rectangle(x0, y0, x1, y1, outline='black', fill='gray')
            # canvas.tag_bind(cell, '<Button-1>', left_click)
            # canvas.tag_bind(cell, '<Button-3>', right_click)
            cells[row][col] = cell
            # print(cells[row][col], cell)

    flatten_board = my_board.reshape(-1)
    for i in range(20):
        n = random.randint(0,cols*rows-1)
        flatten_board[n] = True
    return my_board, cells, canvas

my_board, cells, canvas = make_board()
my_board_a, cells_a, canvas_a = make_board()

color_dicts = {
    0 : "red",
    1 : "black",
    2 : "blue",
    3 : "green",
}
my_board[end_pos] = False
my_board_a[end_pos] = False

def dijkstra_run():
    # for visit_map in astar((0,0), end_pos, my_board):
    for visit_map in dijkstra((0,0), end_pos, my_board):
        for r,cellrows in enumerate(cells):
            for c,cell in enumerate(cellrows):
                current_color_code = 1 if visit_map[r,c] else 0
                if my_board[r,c]:
                    current_color_code = 2
                if end_pos[0] == r and end_pos[1] == c:
                    current_color_code = 3
                current_color = color_dicts[current_color_code]
                canvas.itemconfig(cell, fill=current_color)
        yield

dijkstra_gen = dijkstra_run()

def run_dijkstra_gen():
    try:
        next(dijkstra_gen)
        threading.Timer(0.1,run_dijkstra_gen).start()
    except StopIteration:
        print("dijkstra end")
    

def astar_run():
    for visit_map in astar((0,0), end_pos, my_board_a):
    # for visit_map in dijkstra((0,0), end_pos, my_board):
        for r,cellrows in enumerate(cells_a):
            for c,cell in enumerate(cellrows):
                current_color_code = 1 if visit_map[r,c] else 0
                if my_board_a[r,c]:
                    current_color_code = 2
                if end_pos[0] == r and end_pos[1] == c:
                    current_color_code = 3
                current_color = color_dicts[current_color_code]
                canvas_a.itemconfig(cell, fill=current_color)
        yield

astar_gen = astar_run()
def run_astar_gen():
    try:
        next(astar_gen)
        threading.Timer(0.1,run_astar_gen).start()
    except StopIteration:
        print("astar end")


run_dijkstra_gen()
run_astar_gen()

root.mainloop()

astar end


Exception in thread Thread-342:
Traceback (most recent call last):
  File "c:\Users\c\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\c\anaconda3\lib\threading.py", line 1306, in run
    self.function(*self.args, **self.kwargs)
  File "C:\Users\c\AppData\Local\Temp\ipykernel_8108\685584406.py", line 69, in run_dijkstra_gen
  File "C:\Users\c\AppData\Local\Temp\ipykernel_8108\685584406.py", line 62, in dijkstra_run
  File "c:\Users\c\anaconda3\lib\tkinter\__init__.py", line 2903, in itemconfigure
    return self._configure(('itemconfigure', tagOrId), cnf, kw)
  File "c:\Users\c\anaconda3\lib\tkinter\__init__.py", line 1636, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
RuntimeError: main thread is not in main loop
